# Emulation Exercises

-----------
## Installation 
We use the [Python library](https://python-control.readthedocs.io/en/0.9.3.post2/) `control`, which can be installed using `pip`. If you have no experience with Python, try to do some tutorials (e.g. check [this](https://docs.python.org/3/tutorial/) one). The same goes for installing Python packages using `pip`, see this [tutorial](https://packaging.python.org/en/latest/tutorials/installing-packages/). There are plenty of other Python tutorials for beginners if you do a Google/YouTube search. 

In [1]:
# Install the required python library with pip 
!pip install control


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


------
## Python Packages
We use the following Python libraries which need to be imported. If you have no experience with the [NumPy](https://numpy.org/) library, read the documentation and do some tutorials. It is very important for matrix operations in Python.

In [2]:
# Import the required python libraries
from typing import Optional, List
import numpy as np
import matplotlib.pyplot as plt
import control as ct

## System Library

In here you will find some different systems that show the advatages and disadvantages for the three discretization methods.

In [9]:
# Euler forward example
def create_system1():
    A = np.array([[0, 1],
                 [0, 0]])
    B = np.array([[0],
                 [1]])
    C = np.array([[1, 0]])
    D = np.array([[0]])
    return ct.StateSpace(A, B, C, D)

# Euler backwards example
def create_system2():
    A = np.array([[0, 0.1],
                 [0, 0]])
    B = np.array([[0],
                 [1]])
    C = np.array([[1, 0]])
    D = np.array([[0]])
    return ct.StateSpace(A, B, C, D)

# Tustin for now does not need an example, because it will always map a stable pole in c.t. to stable poles in d.t.

#usage
forward_system = create_system1()
backward_system = create_system2()

print("System used for demonstrating euler forward:\n")
print(forward_system)
print("System used for demonstrating euler backward:\n")
print(backward_system)
    

System used for demonstrating euler forward:

A = [[0. 1.]
     [0. 0.]]

B = [[0.]
     [1.]]

C = [[1. 0.]]

D = [[0.]]

System used for demonstrating euler backward:

A = [[0.  0.1]
     [0.  0. ]]

B = [[0.]
     [1.]]

C = [[1. 0.]]

D = [[0.]]



-------
## Implementation of Descretization Methods

### Euler Forward Method

add explanation

In [10]:
def euler_forward(system, dt):
    """
    Discretize a continuous-time system using the Euler forward method.

    Parameters:
    system: State-space system (instance of control.StateSpace).
    dt: Time step for discretization.

    Returns:
    Ad, Bd, Cd, Dd: Discretized state-space matrices.
    """
    A, B, C, D = system.A, system.B, system.C, system.D
    I = np.eye(A.shape[0])
    Ad = I + dt * A
    Bd = dt * B
    Cd = C
    Dd = D
    return ct.StateSpace(Ad, Bd, Cd, Dd)

(array([[1. , 0.1],
       [0. , 1. ]]), array([[0. ],
       [0.1]]), array([[1., 0.]]), array([[0.]]))


### Euler Backwards Method

add explanation

In [15]:
def euler_backward(system, dt):
    """
    Discretize a continuous-time system using the Euler backward method.

    Parameters:
    system: State-space system (instance of control.StateSpace).
    dt: Time step for discretization.

    Returns:
    Ad, Bd, Cd, Dd: Discretized state-space matrices.
    """
    A, B, C, D = system.A, system.B, system.C, system.D
    I = np.eye(A.shape[0])
    Ad = np.linalg.inv(I - dt * A)
    Bd = np.dot(Ad, dt * B)
    Cd = C
    Dd = D
    return ct.StateSpace(Ad, Bd, Cd, Dd)

#test
test = euler_backward(forward_system, 0.1)
print(test)

A = [[1.  0.1]
     [0.  1. ]]

B = [[0.01]
     [0.1 ]]

C = [[1. 0.]]

D = [[0.]]



### Tustin's Method

add explanation

In [17]:
def tustin_method(system, dt):
    """
    Discretize a continuous-time system using Tustin's method (bilinear transformation).
    
    Parameters:
    A, B, C: State-space matrices of the continuous-time system.
    dt: Time step for discretization.
    
    Returns:
    Ad, Bd, Cd: Discretized state-space matrices.
    """
    A, B, C, D = system.A, system.B, system.C, system.D
    I = np.eye(A.shape[0])
    pre_matrix = np.linalg.inv(I - (dt / 2) * A)
    Ad = np.dot(pre_matrix, I + (dt / 2) * A)
    Bd = np.dot(pre_matrix, dt * B)
    Cd = C
    Dd = D
    return ct.StateSpace(Ad, Bd, Cd, Dd)

test
test = tustin_method(forward_system, 0.1)
print(test)

A = [[1.  0.1]
     [0.  1. ]]

B = [[0.005]
     [0.1  ]]

C = [[1. 0.]]

D = [[0.]]



-------
## Exercise 1: Stability Analysis with Eigenvalue Plots

Objective: Analyze and compare the stability of the Duckiebot steering system using different discretization methods.

Tasks:

	1.Discretize the system using Euler forward, Euler backward, and Tustin’s methods for a given time step (e.g., dt = 0.1 seconds).

	2.Calculate the eigenvalues of the Ad matrix for each method.
	
	3.Plot these eigenvalues on the complex plane. Stable systems will have eigenvalues with magnitudes less than 1.

In [19]:
dt = 0.1  # Time step

# Discretize using each method
dt_forward = euler_forward(forward_system, dt)
dt_backward = euler_backward(forward_system, dt)
dt_tustin = tustin_method(forward_system, dt)

# Function to plot eigenvalues
def plot_eigenvalues(system, method_name):
    Ad = system.A
    eigenvalues = np.linalg.eigvals(Ad)
    plt.scatter(eigenvalues.real, eigenvalues.imag, label=method_name)

# Plot eigenvalues
plt.figure(figsize=(8, 6))
plot_eigenvalues(dt_forward, 'Euler Forward')
plot_eigenvalues(dt_backward, 'Euler Backward')
plot_eigenvalues(dt_tustin, 'Tustin')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.xlabel('Real Part')
plt.ylabel('Imaginary Part')
plt.title('Eigenvalues of Discretized Systems')
plt.legend()
plt.grid(True)
plt.show()

AttributeError: 'tuple' object has no attribute 'A'

<Figure size 800x600 with 0 Axes>